# Análisis de datos

Para generar las slides use:

In [30]:
%%bash
jupyter nbconvert ALL_inspire.ipynb --to slides --reveal-prefix "https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.1.0"

[NbConvertApp] Converting notebook ALL_inspire.ipynb to slides
[NbConvertApp] Writing 273514 bytes to ALL_inspire.slides.html


In [87]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth',200)

Download from [here](https://drive.google.com/file/d/1NtocdsxbTky5uLmKp3eDoGcQG7qsTEYv/view?usp=sharing) 

In [5]:
df=pd.read_json('hep_records.json',lines=True)

In [29]:
df.shape

(1271720, 10)

In [367]:
dfr=df[df.references.map(len)>0].reset_index(drop=True)

In [368]:
dfr=dfr.sample(1000).reset_index(drop=True)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [43]:
i=20
dfr.loc[i,'references']

[783169, 370019, 817606, 1187563, 834316, 708944, 784952, 839772, 764799]

In [ ]:
ps.map(lambda x: lv.ratio(x,parameter))

In [307]:
def check_ref(x,ref):
    return x in ref

In [47]:
dfr[dfr.references.map(lambda x: check_ref(x,783169))]

,abstract,authors,citations,co-authors,creation_date,free_keywords,recid,references,standardized_keywords,title
20,A model is presented here that is capable of s...,"[Shahmoradi, Amir]","[1639400, 1455762]",[],2013-08-05,[],1246726,"[783169, 370019, 817606, 1187563, 834316, 7089...",[],Gamma-Ray bursts: Energetics and Prompt Correl...


build relations

In [50]:
dfr=dfr.append( df[df.references.map(lambda x: check_ref(x,783169))] ).reset_index(drop=True)

In [53]:
def check_intersect1d(x,y):
    return np.intersect1d(x,y)

In [52]:
i=20
dfr.loc[i,'references']

[783169, 370019, 817606, 1187563, 834316, 708944, 784952, 839772, 764799]

In [58]:
np.equal?

In [75]:
dfr=dfr.drop_duplicates(['title']).reset_index(drop=True)

In [76]:
dfr[ dfr.references.astype(str)==str(dfr.loc[i,'references']) ]

,abstract,authors,citations,co-authors,creation_date,free_keywords,recid,references,standardized_keywords,title
20,A model is presented here that is capable of s...,"[Shahmoradi, Amir]","[1639400, 1455762]",[],2013-08-05,[],1246726,"[783169, 370019, 817606, 1187563, 834316, 7089...",[],Gamma-Ray bursts: Energetics and Prompt Correl...


In [91]:
dfr['intersect']=dfr.references.map(lambda x: check_intersect1d(x,dfr.loc[i,'references'])).map(len)

In [83]:
i

20

In [89]:
dfr.loc[i,'title']

'Gamma-Ray bursts: Energetics and Prompt Correlations'

In [97]:
dfr.loc[i,'recid']

1246726

In [95]:
rslt=dfr[dfr.intersect>4].drop( dfr.index[[i]] )
rslt['node']=dfr.loc[i,'recid']

In [98]:
rslt['node']=dfr.loc[i,'recid']

In [99]:
rslt

,abstract,authors,citations,co-authors,creation_date,free_keywords,recid,references,standardized_keywords,title,intersect,node
1016,"We calculate durations and spectral parameters for 207 Swift bursts detected by the BAT instrument from April 2007 to August 2009, including 67 events with measured redshifts. This is the first su...","[Butler, Nathaniel R.]","[1298947, 1246726, 1193486, 884753, 1391635, 1357845, 1224726, 838916, 1631771, 1358366, 901151, 1351720, 839772, 1347114, 1321515, 842284, 1358897, 1358898, 1352762, 1340987, 1235004, 1467975, 12...","[Bloom, Joshua S., Poznanski, Dovi]",2009-10,"[gamms rays: bursts, methods: statistical, Gamma-rays: general]",834316,"[495617, 813313, 825357, 651281, 766484, 782869, 709144, 332317, 677410, 819237, 655793, 820785, 495670, 784952, 666682, 575551, 708161, 575556, 722501, 628295, 793165, 820311, 715870, 496735, 139...","[gamma ray: burst, correlation, luminosity: redshift, statistical analysis, trigger: threshold, X-ray, detector: satellite, experimental results, numerical calculations: Monte Carlo]","The Cosmic Rate, Luminosity Function and Intrinsic Correlations of Long GRBs",5,1246726
1043,"It is proposed that the luminosity function, the rest-frame spectral correlations, and distributions of cosmological long-duration (Type-II) gamma-ray bursts (LGRBs) may be very well described as ...","[Shahmoradi, Amir]","[1512896, 1636736, 1377922, 1246726, 1420108, 1262281, 1318315, 1237644, 1639400, 1625715, 1265046, 1334874, 1305404]",[],2012-09,"[dark energy, gamma-ray burst: general, methods: statistical]",1187563,"[723970, 913417, 243724, 87981, 651281, 496658, 847379, 502297, 332317, 668209, 855092, 946741, 784952, 575556, 782917, 839772, 445537, 574050, 884753, 752755, 573053, 519819, 830609, 1091224, 689...",[],A Multivariate Fit Luminosity Function and World Model for Long GRBs,8,1246726
1062,"Gamma-ray bursts (GRBs) are the most luminous electromagnetic explosions in the Universe, which emit up to $8.8\times10^{54}$ erg isotropic equivalent energy in the hard X-ray band. The high lumin...","[Wang, F.Y.]","[1618317, 1395214, 1495951, 1645717, 1380119, 1636120, 1489306, 1504923, 1415709, 1358366, 1489059, 1637550, 1599804, 1474109, 1493826, 1427267, 1649997, 1510224, 1510226, 1510227, 1501454, 151023...","[Dai, Z.G., Liang, E.W.]",2015-04-02,[],1357858,"[611328, 857601, 1298947, 1115140, 835589, 864857, 415751, 733193, 781836, 822530, 686095, 712720, 651281, 713746, 698627, 842284, 699925, 675350, 681495, 709144, 720409, 1224726, 660508, 841306, ...",[],Gamma-ray Burst Cosmology,6,1246726


In [18]:
dff=df.read_json('')

,abstract,authors,citations,co-authors,creation_date,free_keywords,recid,references,standardized_keywords,title
0,Introduction Part I. Isoclinic nn-planes in E2...,"[Wong, Yung-Chow]","[452267, 1520868, 1614833, 275304, 524107, 762...",[],1961,[],1,[],[],"Isoclinic N planes in Euclidean 2N space, Clif..."
1,,"[Breymayer, K.E.]",[],"[Mallory, Kenneth B.]",1963-03,[],17,[],[],SYSTEM INTERCONNECTION AND PROTECTION OF THE M...
2,,"[Johnston, Lawrence II.]",[1374812],"[Murray, Joseph J., Gearhart, Roger A., Hauser...",1977-08,[],18,[48647],[],A target beam monitor using secondary electron...
3,,"[Muray, J.J.]","[1193744, 1320713]",[],1963-06,[],20,[],[],IRRADIATION OF A CERIUM - CONTAINING SILICATE ...
4,,"[Mozley, Robert F.]",[],[],1963-08,[],23,[],[],PROPOSAL FOR A LARGE SOLID ANGLE PHOTON AND EL...
5,,"[Vakhrushin, Yu.P.]",[],"[Ryabov, V.V.]",1977,[],24,[],[],Study of electron Induced Desorption from the ...
6,,"[Miller, Roger H.]",[322667],[],1963-08,[],25,[],[],PROPOSED BUNCH MONITOR
7,,[],[899865],[],1977,[],26,[],[],withdrawn or canceled
8,,[],"[1208691, 1322317]",[],1977,[],27,[],[],withdrawn or canceled
9,,"[Helm, Richard H.]",[],[],1963-06,[],28,[],[],CALCULATIONS ON CYLINDRICAL METAL VACUUM CHAMB...


In [25]:
df.to_excel('kk.xlsx',index=False)

In [28]:
dfx=pd.read_excel('kk.xlsx')

AssertionError: 

In [173]:
n=13
df=pd.DataFrame({'recid':np.arange(1,n+1)  })
l=[]
for i in df.index:
    #print(i)
    l.append(np.random.randint(1,df.recid.max(),np.random.randint(1,df.recid.max())))
    
df['references']=l
df['references']=df.references.map(np.unique)
df

,recid,references
0,1,"[5, 8]"
1,2,"[1, 3, 4, 5, 7, 8, 9, 10, 11]"
2,3,"[3, 5, 7, 8, 9, 10, 12]"
3,4,[2]
4,5,"[1, 3, 6, 11]"
5,6,"[2, 4, 5, 7, 9, 10, 11, 12]"
6,7,"[1, 2, 3, 4, 10, 11, 12]"
7,8,"[2, 4, 5, 6, 7, 8, 11]"
8,9,[1]
9,10,"[1, 3, 4, 8, 10]"


In [ ]:
def check_intersect1d(x,y):
    return np.intersect1d(x,y)

In [178]:
df3

,recid,references
0,1,"[5, 8]"
1,2,"[1, 3, 4, 5, 7, 8, 9, 10, 11]"
3,4,[2]
4,5,"[1, 3, 6, 11]"
5,6,"[2, 4, 5, 7, 9, 10, 11, 12]"
6,7,"[1, 2, 3, 4, 10, 11, 12]"
7,8,"[2, 4, 5, 6, 7, 8, 11]"
8,9,[1]
9,10,"[1, 3, 4, 8, 10]"
10,11,"[3, 4, 5, 6, 7, 10, 11]"


In [207]:
df

,recid,references
0,1,"[5, 8]"
1,2,"[1, 3, 4, 5, 7, 8, 9, 10, 11]"
2,3,"[3, 5, 7, 8, 9, 10, 12]"
3,4,[2]
4,5,"[1, 3, 6, 11]"
5,6,"[2, 4, 5, 7, 9, 10, 11, 12]"
6,7,"[1, 2, 3, 4, 10, 11, 12]"
7,8,"[2, 4, 5, 6, 7, 8, 11]"
8,9,[1]
9,10,"[1, 3, 4, 8, 10]"


In [208]:
df_not_i

,target,references,intersect,source
1,2,"[1, 3, 4, 5, 7, 8, 9, 10, 11]","[5, 8]",1
2,3,"[3, 5, 7, 8, 9, 10, 12]","[5, 8]",1
5,6,"[2, 4, 5, 7, 9, 10, 11, 12]",[5],1
7,8,"[2, 4, 5, 6, 7, 8, 11]","[5, 8]",1
9,10,"[1, 3, 4, 8, 10]",[8],1
10,11,"[3, 4, 5, 6, 7, 10, 11]",[5],1
11,12,"[2, 3, 5, 6, 10, 12]",[5],1


In [357]:
np.stack(( df_not_i.source.values,df_not_i.target.values),axis=1 ).tolist()+[]

[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]

In [373]:
df

,recid,references
0,1,"[5, 8]"
1,2,"[1, 3, 4, 5, 7, 8, 9, 10, 11]"
2,3,"[3, 5, 7, 8, 9, 10, 12]"
3,4,[2]
4,5,"[1, 3, 6, 11]"
5,6,"[2, 4, 5, 7, 9, 10, 11, 12]"
6,7,"[1, 2, 3, 4, 10, 11, 12]"
7,8,"[2, 4, 5, 6, 7, 8, 11]"
8,9,[1]
9,10,"[1, 3, 4, 8, 10]"


In [396]:
def check_not_ref(x,ref):
    return x not in ref

df=df.sort_values('recid').reset_index(drop=True)

ed=pd.DataFrame()
edges=[]

In [397]:
#DEBUG========
if True:
    i=0
for i in df.index:
#=============    
    #remove i
    df_not_i=df.drop(df.index[[i]])
    df_not_i['intersect']=df_not_i.references.map(lambda x: check_intersect1d(x,df.loc[i,'references']))
    df_not_i=df_not_i[df_not_i.intersect.map(len)>0]
    df_not_i=df_not_i.rename_axis({'recid':'target'}, axis = 'columns')
    df_not_i['source']=df.loc[i,'recid']
    # switch source, target order and remove if already in edges
    df_not_i=df_not_i[ ( df_not_i.target.map( lambda x: [x] )+df_not_i.source.map( lambda x: [x] )
          ).map(lambda x: check_not_ref(x,edges) )]        
    # list of edges: [ [source, target],... ]
    edges=edges+np.stack(( df_not_i.source.values,df_not_i.target.values),axis=1 ).tolist()
    #Store Edges DataFrame
    ed=ed.append( df_not_i[['source','target','intersect']] ).reset_index(drop=True)

ed[:3]

,source,target,intersect
0,1,2,"[5, 8]"
1,1,3,"[5, 8]"
2,1,6,[5]


In [398]:
ed[ed.source==2]

,source,target,intersect
7,2,3,"[3, 5, 7, 8, 9, 10]"
8,2,5,"[1, 3, 11]"
9,2,6,"[4, 5, 7, 9, 10, 11]"
10,2,7,"[1, 3, 4, 10, 11]"
11,2,8,"[4, 5, 7, 8, 11]"
12,2,9,[1]
13,2,10,"[1, 3, 4, 8, 10]"
14,2,11,"[3, 4, 5, 7, 10, 11]"
15,2,12,"[3, 5, 10]"
16,2,13,[11]


Fix edge

In [400]:
edf=ed[ed.intersect.map(len)>3].reset_index(drop=True)
edf

,source,target,intersect
0,2,3,"[3, 5, 7, 8, 9, 10]"
1,2,6,"[4, 5, 7, 9, 10, 11]"
2,2,7,"[1, 3, 4, 10, 11]"
3,2,8,"[4, 5, 7, 8, 11]"
4,2,10,"[1, 3, 4, 8, 10]"
5,2,11,"[3, 4, 5, 7, 10, 11]"
6,3,6,"[5, 7, 9, 10, 12]"
7,3,11,"[3, 5, 7, 10]"
8,3,12,"[3, 5, 10, 12]"
9,6,7,"[2, 4, 10, 11, 12]"


In [401]:
df.to_csv('example_colegios.csv',index=False)

In [ ]:
# 
red               colegio 1             colegio 2
2
3,6,7,8,10,11 -> (2,3,6,7,8,10,11)      desconectados: 4,5,9
3 in colegio:
    True 
    6,11 True

    12 False -> (2,3,6,7,8,10,11,12)
6 in colegio:
    True
    7,8,11,12 True
7  in colegio:
    True:
    3,6,10,11,12 True

8 in colegio:
    True:
    11,12
    
... 
15 en colegio:
    False
    start colegio 3

In [377]:
if True:
    i=2
    #remove i=2
    df_not_i=df.drop(df.index[[i]])
    df_not_i['intersect']=df_not_i.references.map(lambda x: check_intersect1d(x,df.loc[i,'references']))
    df_not_i=df_not_i[df_not_i.intersect.map(len)>0]
    df_not_i=df_not_i.rename_axis({'recid':'target'}, axis = 'columns')
    df_not_i['source']=df.loc[i,'recid']
    # switch source, target order and remove if already in edges
    df_not_i=df_not_i[ ( df_not_i.target.map( lambda x: [x] )+df_not_i.source.map( lambda x: [x] )
          ).map(lambda x: check_not_ref(x,edges) )]    
    
    # list of edges: [ [source, target],... ]
    edges=edges+np.stack(( df_not_i.source.values,df_not_i.target.values),axis=1 ).tolist()    
    #Store Edges DataFrame
    ed=ed.append( df_not_i[['source','target','intersect']] ).reset_index(drop=True)

ed    

,source,target,intersect
0,1,2,"[5, 8]"
1,1,3,"[5, 8]"
2,1,6,[5]
3,1,8,"[5, 8]"
4,1,10,[8]
5,1,11,[5]
6,1,12,[5]
7,2,3,"[3, 5, 7, 8, 9, 10]"
8,2,5,"[1, 3, 11]"
9,2,6,"[4, 5, 7, 9, 10, 11]"


In [366]:
edges

[[1, 2],
 [1, 3],
 [1, 6],
 [1, 8],
 [1, 10],
 [1, 11],
 [1, 12],
 [2, 3],
 [2, 5],
 [2, 6],
 [2, 7],
 [2, 8],
 [2, 9],
 [2, 10],
 [2, 11],
 [2, 12],
 [2, 13]]

,target,references,intersect,source,ST
2,3,"[3, 5, 7, 8, 9, 10, 12]","[3, 5, 7, 8, 9, 10]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
4,5,"[1, 3, 6, 11]","[1, 3, 11]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
5,6,"[2, 4, 5, 7, 9, 10, 11, 12]","[4, 5, 7, 9, 10, 11]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
6,7,"[1, 2, 3, 4, 10, 11, 12]","[1, 3, 4, 10, 11]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
7,8,"[2, 4, 5, 6, 7, 8, 11]","[4, 5, 7, 8, 11]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
8,9,[1],[1],2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
9,10,"[1, 3, 4, 8, 10]","[1, 3, 4, 8, 10]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
10,11,"[3, 4, 5, 6, 7, 10, 11]","[3, 4, 5, 7, 10, 11]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
11,12,"[2, 3, 5, 6, 10, 12]","[3, 5, 10]",2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"
12,13,[11],[11],2,"[[1, 2], [1, 3], [1, 6], [1, 8], [1, 10], [1, 11], [1, 12]]"


In [318]:
(df_not_i.target.astype(str)+'::'+df_not_i.source.astype(str) ).str.split('::')

SyntaxError: unexpected EOF while parsing (<ipython-input-318-823e03b869c7>, line 1)

In [312]:
( df_not_i.target.astype(str)+'::'+df_not_i.source.astype(str) ).str.split('::').values

array([list(['1', '2']), list(['3', '2']), list(['5', '2']),
       list(['6', '2']), list(['7', '2']), list(['8', '2']),
       list(['9', '2']), list(['10', '2']), list(['11', '2']),
       list(['12', '2']), list(['13', '2'])], dtype=object)

In [308]:
def check_ref(x,ref):
    return x in ref
( df_not_i.target.astype(str)+'::'+df_not_i.source.astype(str) ).str.split('::').map(lambda x: check_ref(x,[[1,2]] ))
           #np.stack(( ed.source.values,ed.target.values),axis=1 )          ))

0     False
2     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
dtype: bool

In [310]:
check_ref([1,2] , [[1,2]])

True

In [287]:
df_not_i.assign(  TS= pd.Series( {'TS':np.stack(( df_not_i.target.values,df_not_i.source.values),axis=1 )}).values )

ValueError: Length of values does not match length of index

In [242]:
1+1

2

In [240]:
np.stack?